# Notebook and python basics

The jupyter notebook is an interactive python environment that enables users to enter and execute commands in "cells," much like the matlab interactive environment.  We use it primarily as a teaching tool, since live coding is much easier to follow in the notebook and markdown documentation can be inserted inline.  Here, we cover a few basic features of the notebook environment and python before we begin with any of the materials science.

To execute a cell, you can either click the "play" button in the toolbar, or press `shift+enter` while the cell is active.  For example:

In [ ]:
print("Hello world!")

In [ ]:
a = 3
b = 4
print(a + b)

In [ ]:
list_of_numbers = [1, 2, 3]
list_sum = 0
# A basic for loop
for number in list_of_numbers:
    list_sum = list_sum + number

print("Sum of list is", list_sum)

Note that `print` is a *function* for which the arguments are enclosed in parentheses.  We'll be using a variety of functions in this lesson.  Note that you can import functions from libraries and modules you have installed.  For example, from the math library, you might want to import the sine function:

In [ ]:
from math import cos

You can find information about functions and modules via the following commands in the jupyter notebook.  Use the built-in function `help` to print information about a function or object.  Using the question mark will print a tooltip at the bottom of the screen.  I tend to use the `shift+tab` command while the cursor is inside of a function call's parentheses most regularly.

In [ ]:
help(cos)
cos?
cos(5) # press shift+tab with the cursor inside the parentheses

Python has a rich ecosystem of packages, of which pymatgen is one, that are accessible via both the conda and python package index software managers.  Some of the most popular include **numpy** (for matrix/vector numerical operations), **scipy** (for various scientific computing tools, including ODEs, numerical optimization), **matplotlib** (for plotting), and **pandas** (for dataframe-based analysis akin to R).  The following links might be useful, if you're interested in learning more about python or scientific computing using python generally.

* [PyPI - the Python Package Index](https://pypi.python.org/pypi)
* [Scipy and Numpy documentation](https://docs.scipy.org/doc/)
* [Matplotlib examples](http://matplotlib.org/gallery.html)
* [A gallery of interesting notebooks](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks)


# Pymatgen core functionality

Pymatgen is the code that powers all of the analysis that's used in the Materials Project.  It includes a robust and efficient libraries for the handling of structures and molecules, in addition to various mathematical and scientific tools for the handling and generation of materials data.  Here are a few things you can do with pymatgen:

- Create, identify, and manipulate crystal structures and molecules
- Write input and output files for most electronic structure codes
- Analyze density of states, XRD spectra, and bandstructure data
- Tensor-based analysis, including Elastic and Piezoelectric tensors
- Analysis of the local chemical environment of structural sites
- Create pourbaix and phase diagrams
- Match substrates based on geometry and elastic behavior
- Create and manipulate surfaces
- Do unit conversions
- Get basic information about chemical identity
- Estimate the cost of a material based on chemical abundance

## Structures, sites, and lattices

Most of the fundamentals of pymatgen are expressed in the Structure and Lattice objects.  These objects contain data on the lattice parameters and the location of individual sites within lattices.  Let's start by importing those objects.

In [ ]:
from pymatgen import Structure, Lattice, Molecule

The general lattice constructor takes a 3x3 array as it's argument, which consists of the vectors that compose the unit cell.  There are also convenience constructors that allow you to construct lattices from lengths and angles, as well as from specific crystal systems with appropriate input parameters.

In [ ]:
# Making lattices
lattice = Lattice([[2.8, 0, 0], [0, 2.8, 0], [0, 0, 2.8]])
lattice = Lattice.from_lengths_and_angles([2.8, 2.8, 2.8], [90, 90, 90])
lattice.cubic(2.8)

lattice.hexagonal(a = 2.8, c = 3.6)
lattice.rhombohedral(a = 2.8, alpha = 60)

# Getting lattice info
print("a = ", lattice.a)
print("alpha = ", lattice.alpha)
print("volume = ", lattice.volume)

Structures objects are lattices with the addition of contained species.  Structures are constructed from a lattice, a list of species, and a list of coordinates that correspond to each species.  Note that species in this string can contain occupancies (and sometimes must in order to use other tools!).  You can also create structures from spacegroups, and from cif files.

In [ ]:
# Making structures
bcc_fe = Structure(lattice, ["Fe", "Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])
site0 = bcc_fe[0]
site0.coords
site0.species_string
site0.x

bcc_fe = Structure.from_spacegroup("Im-3m", Lattice.cubic(2.8), ["Fe"], [[0, 0, 0]])
print(bcc_fe)
nacl= Structure.from_spacegroup("Fm-3m", Lattice.cubic(5.692), ["Na+", "Cl-"], 
                                [[0, 0, 0], [0.5, 0.5, 0.5]])
big_structure = Structure.from_file("Nb2O5.cif")
big_structure.formula

Disordered structures can also be constructed using dictionaries that correspond to the species and its occupancy.

In [ ]:
# Making disordered structures
specie = {"Cu0+": 0.5, "Au0+":0.5}
cu_au = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])
print(cu_au)

You can also assign site properties flexibly, and some site properties, like `selective_dynamics` will be used in other methods, such as writing a file to POSCAR.

In [ ]:
# Manipulating structures and assigning properties to sites
big_structure[0] = "V"
big_structure.formula
big_structure[0] = "Nb"

bcc_fe.append("C", [0.25, 0.25, 0.25])
bcc_fe.pop(-1)
bcc_fe.make_supercell([2, 2, 2])

sd = []
names = []
for site in big_structure:
    if site.species_string == "Nb":
        sd.append([False, False, False])
    else:
        sd.append([True, True, True])
big_structure.add_site_property("selective_dynamics", sd)
big_structure.to(filename="POSCAR")

**Visualization**: note that pymatgen has a visualization package that requires VTK to run.  We also have a notebook viewing tool, which we'll use here.

In [ ]:
from pymatgen.vis.structure_chemview import quick_view

In [ ]:
quick_view(nacl)

### Exercise 1: 

You're studying materials used in the chlor-alkali process for the production of Cl<sub>2</sub>.  Find your favorite oxide using the materials project rester.  Replace each oxygen atom with chlorine.

In [ ]:
from pymatgen import MPRester
mpr = MPRester()
# Potential solution:
structure = Structure.from_file("BaNiO3.cif")
for n in range(structure.num_sites):
    if structure[n].species_string == 'O':
        structure[n] = 'Cl'

# Bonus solution
structure.replace_species({'Cl':'O'})
structure

---------
## Input/Output

Pymatgen has extensive I/O capabilities for both structures and files for electronic structure codes, including a native serialization format for nearly all of its core classes.  This means pymatgen objects can be very easily inserted into a Mongo-style collection, which we'll discuss more later.  Structure and molecule objects can be output into both friendly serialization formats (json, yaml) and a number of standard file formats automatically.  For molecules, this includes any file format supported by OpenBabel, assuming you have that dependency installed.

In [ ]:
bcc_fe.to("bcc_fe.cif")
bcc_fe.to(filename="bcc_fe.cif")
bcc_fe.to(filename="bcc_fe.json")

molecule = Molecule(['C', 'O'], [[0,0,0],[0, 0, 1.2]])
molecule.to(filename="molecule.xyz")

The primary way that pymatgen serializes objects for file output is via another module named "monty" (the hidden complement to python), which has two functions that will put any object with a native `as_dict` method into a json or yaml file.  A number of classes have methods that will output their contents to a file, but some don't, so you can use monty.serialization.dumpfn and loadfn to output any object with a as_dict/from_dict to a file (or read it from a file).  Here are a few examples:

In [ ]:
from monty.serialization import dumpfn, loadfn
dumpfn(bcc_fe, "bcc_fe.json")
new_struct = loadfn("bcc_fe.json")
bandstructure = loadfn("li2o_bs.json")

For electronic structure and MD, pymatgen has interfaces to the following codes and I/O formats:

* abinit
* exciting
* feff
* lammps
* VASP
* adf
* aiida
* ase
* OpenBabel
* cif
* cssr
* fiesta
* gaussian
* nwchem
* phonopy
* pwscf (Quantum Espresso)
* qchem
* xcrysden
* xr
* xyz
* zeopp

Since the majority of the data hosted on the materials project is determined using VASP, we'll walk you through the VASP io module in pymatgen.  Most input and output files in pymatgen have their own associated object, contained in pymatgen.io.vasp.inputs or pymatgen.io.vasp.outputs.  However, the main way in which pymatgen generates input for vasp calculations is via the input sets, contained in pymatgen.io.vasp.sets, which define vasp input parameters for a given calculation equivalent to what has been used to generate data on the materials project.  There are a number of input sets, which each have different parameters, but the main one we use for structure optimization is called the MPRelaxSet.

In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet
vis = MPRelaxSet(nacl)
vis.write_input("vasp_input")

In [ ]:
!ls vasp_input/

The MPRelaxSet has a number of convenient features, including automatically setting MAGMOMS, LDAUU values, and scaling k-points to a density.  Most of the customizations you might add on top of our set can be achieved using user_incar_settings:

In [ ]:
vis = MPRelaxSet(nacl, user_incar_settings={"ISYM": 0})
vis.write_input("vasp_input_2")
!diff vasp_input/INCAR vasp_input_2/INCAR

We're not going to discuss VASP input and output any further because you may not use VASP for your work, and because we'll be covering ways in which you can automate all of this using workflows is tomorrow's morning session, but there are a number of very sophisticated methods for parsing outputs, and a number of other input sets you can use for more specific calculations like HSE bandstructure runs, slab calculations, and native vasp elastic calculations.

### Exercise: Load the structures contained in "li_s_structures.json" and use the MPRelaxSet to write VASP inputs to a set of distinct directories that you could use to run vasp later.

In [ ]:
structures = loadfn('li_s_structures.json')
for n, structure in enumerate(structures):
    vis = MPRelaxSet(structure)
    vis.write_input("vasp_directory_" + str(n))
!ls

---------
## SymmetryAnalyzer and Operations

In addition to bookkeeping of structures using Structure objects, pymatgen contains powerful tools for analyzing crystal symmetry and comparing structures.  The SymmetryAnalyzer object is essentially a wrapper around spglib, which is written in c for more efficient determination of invariant symmetry operations and thus crystal symmetry.  The symmetry analyzer can be used to get primitive and standardized conventional cell settings of structures.

In [ ]:
# Get primitive structure of BCC iron
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
sga = SpacegroupAnalyzer(bcc_fe)
prim = sga.get_primitive_standard_structure() # Note only a single site
sga.get_conventional_standard_structure()
print("Crystal system:", sga.get_crystal_system())
print("Spacegroup symbol:", sga.get_space_group_symbol())

ops = sga.get_space_group_operations()

In [ ]:
pga = PointGroupAnalyzer(molecule)
pga.get_pointgroup()
pga.get_pointgroup()

--------
## XRD, Bandstructure, and Density of States

Pymatgen has various tools that allow for the analysis and plotting of structural and electronic information.  The XRDCalculator is perhaps the most straightforward of these tools, since it only requires a structure object.

In [ ]:
from pymatgen.analysis.diffraction.xrd import XRDCalculator, ATOMIC_SCATTERING_PARAMS
XRDCalculator.AVAILABLE_RADIATION
xrdc = XRDCalculator()
big_xrd = xrdc.get_xrd_data(big_structure)

In [ ]:
%matplotlib inline
xrdc.show_xrd_plot(nacl)

 ### Exercise 3: XRD spectra of Li<sub>x</sub>Si<sub>y</sub>
 
Your experimental collaborator finds an interesting Li-S cathode and performs powder XRD on it, resulting in the spectra below.  Identify the structure using the pymatgen XRD calculator.
![LiS XRD](LiS_XRD.png)


In [ ]:
from IPython.display import Image
Image("LiS_XRD.png")

In [ ]:
lis_structures = loadfn("li_s_structures.json")
data = xrdc.get_xrd_data(lis_structures[0], two_theta_range=[10,80])
for n, lis_structure in enumerate(lis_structures):
    data = xrdc.get_xrd_data(lis_structure, two_theta_range=[10,80])
    if len(data) == 8:
        print(lis_structure)
    xrdc.show_xrd_plot(lis_structure)

### Exercise 4: Electronic structure of TiO<sub>2</sub>

In [ ]:
from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter
from pymatgen.electronic_structure.core import OrbitalType

bs = mpr.get_bandstructure_by_material_id("mp-2657")
print(bs.get_band_gap())
plotter=BSPlotter(bs)
#plotter.get_plot().show()
#plotter.plot_brillouin()

dos = mpr.get_dos_by_material_id("mp-2657")
dp = DosPlotter()
dos_ti = dos.get_element_spd_dos("Ti")
dos_o = dos.get_element_spd_dos("O")
dp.add_dos("O p-states", dos_o[OrbitalType.p])
dp.add_dos("Ti d-states", dos_ti[OrbitalType.d])
#dp.get_plot().show()

## Tensors

### Exercise 5:

Fit a "noisy" tensor to a particular crystal structure

In [ ]:
import json
import numpy as np
from pymatgen.analysis.elasticity.elastic import ElasticTensor
data = json.load(open("sample_elastic.json"))
si_struct = Structure.from_dict(data[0])
et = ElasticTensor.from_voigt(data[1])

print(np.array(data[1]))
et.fit_to_structure(si_struct).voigt.round(2)

## Surfaces

### Generate all of the low-index facets for BCC Fe

In [ ]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import generate_all_slabs
lattice = Lattice.cubic(2.85)
structure = Structure(lattice, ["Fe", "Fe"],
                      [[0, 0, 0], [0.5, 0.5, 0.5]])

slabs = generate_all_slabs(structure, 1, 4, 10)
first_slab = slabs[0]
first_slab.miller_index
for slab in slabs:
    print slab.miller_index

quick_view(slab)

# Summary

This notebook is intended to provide a short introduction to some of the functionality of pymatgen. Pourbaix diagram and phase diagram generation require a bit more in-depth use of the API, and we don't have time to cover that here, but you can find additional examples [here](http://pymatgen.org/examples.html).

In conclusion, I'd like to note that pymatgen development is ongoing and encourage you to get involved.  We have a [git repository](www.github.com/materialsproject/pymatgen), which you can clone/fork to customize the code.I'd encourage anyone interested in computational materials science to get a basic sense of how contributing to an open-source repository works.  You basically create a copy of the repository on github (i. e. a fork), make changes there, and submit them in what's called a pull request.